In [190]:
import sys, os
import numpy as np
import pandas as pd

sys.path.append(os.path.join('..'))
import original_lingam

sys.path.append(os.path.join('..'))
import proposed_lingam

sys.path.append(os.path.join('..'))
import false_lingam

sys.path.append(os.path.join('..', 'sythetic_data'))
import generate
from generate import generate

sys.path.append(os.path.join('..', 'related_methods'))
import notears

sys.path.append(os.path.join('..', 'related_methods'))
import ges

from supplement import criteria
from supplement import threshold

import random
import scipy
import networkx
import seaborn as sns
import matplotlib.pyplot as plt
# import sempler_tool
import supplement
from sklearn.linear_model import LinearRegression

np.set_printoptions(precision=3, suppress=True)
# np.random.seed(2022)

In [191]:
X = generate(20, 3, 4)
X = check_array(X[0])

Ground Truth:
[[ 0 -3 -2]
 [ 0  0 -1]
 [ 0  0  0]]


In [192]:
X

array([[ 0.976, -2.923,  1.658],
       [ 0.779, -2.14 ,  2.428],
       [ 0.164,  0.106, -0.038],
       [ 0.044,  0.824,  0.473],
       [ 0.786, -1.493,  0.168],
       [ 0.601, -1.634,  1.575],
       [ 0.528, -0.646,  0.22 ],
       [ 0.158,  0.071,  0.775],
       [ 0.401, -0.555,  0.773],
       [ 0.767, -2.123,  1.467],
       [ 0.687, -1.806,  1.613],
       [ 0.896, -2.324,  1.362],
       [ 0.498, -1.311,  1.666],
       [ 0.83 , -2.074,  1.723],
       [ 0.331, -0.421,  1.465],
       [ 0.596, -1.702,  1.354],
       [ 0.733, -1.303,  0.956],
       [ 0.065,  0.345,  0.269],
       [ 0.364, -0.323, -0.246],
       [ 0.684, -1.537,  1.585]])

In [193]:
def multiple_residual(xi, xj):
    """The residual when xi is regressed on xj. xi = y, xj = x"""
    model_lr = LinearRegression()
    model_lr.fit(xj, xi)
    res = np.array(xi - model_lr.predict(xj))
    return res

In [194]:
multiple_residual(X[:, 0], X[:, [1, 2]])

array([-0.066,  0.034, -0.08 ,  0.059,  0.069, -0.054,  0.074, -0.035,
        0.017, -0.045, -0.018,  0.014, -0.051,  0.052,  0.04 , -0.096,
        0.133, -0.082, -0.027,  0.06 ])

In [44]:
def residual(xi, xj):
    return xi - (np.cov(xi, xj)[0, 1] / np.var(xj)) * xj

In [116]:
d = 3
def BF_res(d, m, X):
    def residual(xi, xj):
        return xi - (np.cov(xi, xj)[0, 1] / np.var(xj)) * xj
#     res(d, 1, X) = residual(X[:, d-1], X[:, 0])
    if m == 1:
        return residual(X[:, d-1], X[:, 0]) 
    else:
        return residual(BF_res(d, m-1, X), X[:, m-1])

In [127]:
BF_res(3, 3, X)

array([0.741, 1.027, 0.966, 1.345, 1.325, 1.062, 0.739, 0.75 , 1.258,
       0.546])

In [118]:
def MI(x1, x2, param = [2e-2, 1.0]):
        """Calculate the mutual informations."""
        kappa, sigma = param
        n = len(x1)
        X1 = np.tile(x1, (n, 1))
        K1 = np.exp(-1/(2*sigma**2) * (X1**2 + X1.T**2 - 2*X1*X1.T))
        X2 = np.tile(x2, (n, 1))
        K2 = np.exp(-1/(2*sigma**2) * (X2**2 + X2.T**2 - 2*X2*X2.T))

        tmp1 = K1 + n*kappa*np.identity(n)/2
        tmp2 = K2 + n*kappa*np.identity(n)/2
        K_kappa = np.r_[np.c_[tmp1 @ tmp1, K1 @ K2],
                        np.c_[K2 @ K1, tmp2 @ tmp2]]
        D_kappa = np.r_[np.c_[tmp1 @ tmp1, np.zeros([n, n])],
                        np.c_[np.zeros([n, n]), tmp2 @ tmp2]]

        sigma_K = np.linalg.svd(K_kappa, compute_uv=False)
        sigma_D = np.linalg.svd(D_kappa, compute_uv=False)
        return (-1/2)*(np.sum(np.log(sigma_K)) - np.sum(np.log(sigma_D)))

In [132]:
def pre_loss(dimension, d_all, X):
    param = [2e-2, 1.0]
    ans = 0.0
    if dimension == 1:
        for i in range(2, d_all+1):
            ans += MI(X[:, 0], BF_res(i, 1, X), param)
    else:
        for i in range(dimension+1, d_all+1):
            ans += MI(BF_res(dimension, dimension-1, X), BF_res(i, dimension, X), param)
    return ans

In [133]:
def LOSS(d, X):
    res = 0.0
    for i in range(1, d):
        res += pre_loss(i, d, X)
    return res

In [134]:
LOSS(3, X)

4.724320739861547

In [141]:
LOSS(3, colchanged_X)

4.642437657132771

In [136]:
X

array([[ 0.507,  0.245, -0.773],
       [ 0.749,  0.459, -1.445],
       [ 0.367,  0.479, -0.557],
       [ 0.72 ,  0.833, -1.691],
       [ 0.236,  0.894, -0.552],
       [ 0.43 ,  0.765, -1.191],
       [ 0.35 ,  0.302, -0.475],
       [ 0.157,  0.28 ,  0.105],
       [ 0.841,  0.041, -0.458],
       [ 0.583,  0.456, -1.716]])

In [142]:
 X.shape[1]

3

In [140]:
colchanged_X

array([[ 0.245,  0.507, -0.773],
       [ 0.459,  0.749, -1.445],
       [ 0.479,  0.367, -0.557],
       [ 0.833,  0.72 , -1.691],
       [ 0.894,  0.236, -0.552],
       [ 0.765,  0.43 , -1.191],
       [ 0.302,  0.35 , -0.475],
       [ 0.28 ,  0.157,  0.105],
       [ 0.041,  0.841, -0.458],
       [ 0.456,  0.583, -1.716]])

In [ ]:
change_dict = list(itertools.permutations(arr))
for i in range(len(change_dict)):
    for j in range(len(change_dict[i])):
        colchanged_X[:, j] = X[:, change_dict[i][j]]

In [122]:
X = generate(10, 3, 342334)

Ground Truth:
[[ 0  0 -2]
 [ 0  0 -2]
 [ 0  0  0]]


In [123]:
d = 3
import itertools
arr = [i for i in range(d)]
print(list(itertools.permutations(arr)))

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]


In [124]:
from sklearn.preprocessing import scale
from sklearn.utils import check_array
print(type(X[0]))
print(X[0])
X = check_array(X[0])

<class 'pandas.core.frame.DataFrame'>
        x_0       x_1       x_2
0  0.507478  0.245048 -0.772838
1  0.748803  0.458598 -1.444755
2  0.366864  0.478928 -0.556729
3  0.720476  0.833200 -1.691240
4  0.236079  0.894185 -0.552491
5  0.430160  0.765291 -1.191042
6  0.349565  0.302426 -0.474977
7  0.157240  0.279715  0.105294
8  0.840752  0.041361 -0.457780
9  0.583285  0.456202 -1.716221


In [125]:
colchanged_X = np.zeros((10, 3))

In [129]:
X

array([[ 0.507,  0.245, -0.773],
       [ 0.749,  0.459, -1.445],
       [ 0.367,  0.479, -0.557],
       [ 0.72 ,  0.833, -1.691],
       [ 0.236,  0.894, -0.552],
       [ 0.43 ,  0.765, -1.191],
       [ 0.35 ,  0.302, -0.475],
       [ 0.157,  0.28 ,  0.105],
       [ 0.841,  0.041, -0.458],
       [ 0.583,  0.456, -1.716]])

In [138]:
change_dict = list(itertools.permutations(arr))
for j in range(len(change_dict[2])):
    colchanged_X[:, j] = X[:, change_dict[2][j]]

In [139]:
colchanged_X

array([[ 0.245,  0.507, -0.773],
       [ 0.459,  0.749, -1.445],
       [ 0.479,  0.367, -0.557],
       [ 0.833,  0.72 , -1.691],
       [ 0.894,  0.236, -0.552],
       [ 0.765,  0.43 , -1.191],
       [ 0.302,  0.35 , -0.475],
       [ 0.28 ,  0.157,  0.105],
       [ 0.041,  0.841, -0.458],
       [ 0.456,  0.583, -1.716]])

In [130]:
len(X)

10

In [174]:
X = generate(500, 3, 34233)
X = check_array(X[0])

Ground Truth:
[[ 0  1 -1]
 [ 0  0  0]
 [ 0  0  0]]


In [175]:
len(X)

500

In [178]:
def search_causal_order_kernel(X):
    """Search the causal ordering by kernel method."""
#     if len(U) == 1:
#         return U[0]
    Tkernels = []
    colchanged_X = np.zeros((len(X), X.shape[1]))
    d = X.shape[1]
    arr = [i for i in range(d)]
    change_dict = list(itertools.permutations(arr))
    print(change_dict)
    for i in range(len(change_dict)):
        for j in range(len(change_dict[i])):
            colchanged_X[:, j] = X[:, change_dict[i][j]]
        Tkernels.append(LOSS(d, colchanged_X))
    print(Tkernels)
    index = np.argmin(Tkernels)
    return list(change_dict[index])

In [179]:
list(search_causal_order_kernel(X))

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]
[4.582027502028268, 4.582358600762063, 4.47699093812048, 4.454065748929452, 4.492350634937793, 4.462065431749124]


[1, 2, 0]